In [1]:

# === Path kandidat file .safetensors ===
# Ubah PATH di sini kalau perlu
from pathlib import Path

CANDIDATES = [
    "./mobilenet_v2_1_0_imagenet.safetensors",
    "./model.safetensors",
    "./mobilenet_v2_1_0_imagenet.safetensors",
    "./model.safetensors",
]

PATH = None
for c in CANDIDATES:
    if Path(c).exists():
        PATH = c
        break

print("Detected file:", PATH)
if PATH is None:
    print("⚠ File tidak ditemukan di kandidat paths. Set PATH manual.")


Detected file: ./mobilenet_v2_1_0_imagenet.safetensors


In [2]:

# === Import yang dibutuhkan ===
# Prefer numpy loader agar tidak perlu PyTorch untuk sekadar inspeksi.
try:
    from safetensors.numpy import load_file as load_np
except Exception as e:
    raise SystemExit(
        "Tidak bisa import safetensors.numpy. Install dulu:\n"
        "  pip install safetensors\n"
        f"Detail error: {e}"
    )

import pandas as pd
import re, math
from collections import defaultdict


In [3]:

# === Baca file & list keys ===
if PATH is None:
    raise SystemExit("Set PATH ke file .safetensors yang benar lalu jalankan lagi.")

sd = load_np(PATH)   # dict-like: key -> numpy array
print(f"Loaded tensors: {len(sd)}")

# Buat DataFrame index keys
rows = []
total_params = 0
for k, arr in sd.items():
    shape = tuple(arr.shape)
    dtype = str(arr.dtype)
    numel = int(arr.size)
    total_params += numel
    rows.append({"key": k, "shape": shape, "dtype": dtype, "numel": numel})

df = pd.DataFrame(rows).sort_values("key").reset_index(drop=True)
print("Total parameters (numel):", total_params)
df.head(20)


Loaded tensors: 314
Total parameters (numel): 3539036


,key,shape,dtype,numel
0,blocks.0.0.bn1.bias,"(32,)",float32,32
1,blocks.0.0.bn1.num_batches_tracked,(),int64,1
2,blocks.0.0.bn1.running_mean,"(32,)",float32,32
3,blocks.0.0.bn1.running_var,"(32,)",float32,32
4,blocks.0.0.bn1.weight,"(32,)",float32,32
5,blocks.0.0.bn2.bias,"(16,)",float32,16
6,blocks.0.0.bn2.num_batches_tracked,(),int64,1
7,blocks.0.0.bn2.running_mean,"(16,)",float32,16
8,blocks.0.0.bn2.running_var,"(16,)",float32,16
9,blocks.0.0.bn2.weight,"(16,)",float32,16


In [4]:

# === Lihat prefix unik untuk memahami skema penamaan (TIMM biasanya: conv_stem, bn1, blocks.X.conv.*, conv_head, bn2, classifier) ===
def first_tokens(s, n=2):
    toks = s.split(".")
    return ".".join(toks[:n]) if len(toks) >= n else s

df["prefix2"] = df["key"].map(lambda s: first_tokens(s, 2))
df["prefix3"] = df["key"].map(lambda s: first_tokens(s, 3))

print("Prefix level-2:")
display(df["prefix2"].value_counts().to_frame("count"))

print("\nPrefix level-3 (berguna untuk melihat 'blocks.N.conv'):")
display(df["prefix3"].value_counts().to_frame("count").head(30))


Prefix level-2:


,count
prefix2,
blocks.3,72
blocks.2,54
blocks.4,54
blocks.5,54
blocks.1,36
blocks.6,18
blocks.0,12
bn2.running_mean,1
conv_head.weight,1



Prefix level-3 (berguna untuk melihat 'blocks.N.conv'):


,count
prefix3,
blocks.5.2,18
blocks.6.0,18
blocks.1.1,18
blocks.2.0,18
blocks.2.1,18
blocks.2.2,18
blocks.3.0,18
blocks.3.1,18
blocks.3.2,18
